# Build an personal trainer with Meta Llama 4 using watsonx.ai
**Authors:** Ash Minhas, Anna Gutowska

In this tutorial, you will build a generative AI-powered personal trainer. This AI trainer leverages the latest opensource [Meta Llama 4 Scout](https://www.llama.com/) [large language model (LLM)](https://www.ibm.com/think/topics/large-language-models) for processing image input and generating personalized workout plans to reach your fitness goals effectively. We will access the model through the [IBM watsonx.ai™](https://www.ibm.com/products/watsonx-ai) API. 

## Introduction
Are you looking to be more active? Has your fitness journey reached a plateau and are you looking to take your workout routines to the next level? In this tutorial, you will use [AI](https://www.ibm.com/think/topics/artificial-intelligence) to provide a personalized training experience. 

## AI fitness: How it works
Our workout app is composed of the following stages:

1. The user uploads images of their current workout equipment, one item at a time. This set can consist of both home & gym equipment. 
2. The user selects the following criteria:   
    * Workout type: strength, endurance, flexibility, balance, cardio and so on.
    * Workout length: 30 minutes, 1 hour, 1.5 hours and so on.
    * Fitness level: beginner, intermediate or advanced. 
3. Upon submission of the input, the multimodal Llama 4 Scout model iterates over the list of images and returns the following output:
    * Description of the item: either “equipment” or “other.”
    * Category: "barbell", "dumbbell", "machine", "bodyweight" and so on.
    * Workout type: "strength", "endurance", "flexibility", "balance", "cardio" and so on.

4. The same Llama 4 model then serves as a fitness coach. The LLM uses the previous output to provide a training plan that is suitable for the user’s selections.

5. The training program and the images in the described personalized recommendation are all returned to the user.

# Prerequisites

You need an [IBM Cloud® account](https://cloud.ibm.com/registration) to create a [watsonx.ai™](https://www.ibm.com/products/watsonx-ai) project.

# Steps

In order to use the watsonx® [application programming interface (API)](https://www.ibm.com/think/topics/api), you will need to complete the following steps.

## Step 1. Set up your environment

1. Log in to [watsonx.ai](https://dataplatform.cloud.ibm.com/registration/stepone?context=wx&apps=all) by using your IBM Cloud account.

2. Create a [watsonx.ai project](https://www.ibm.com/docs/en/watsonx/saas?topic=projects-creating-project).

	You can get your project ID from within your project. Click the **Manage** tab. Then, copy the project ID from the **Details** section of the **General** page. You need this ID for this tutorial.

## Step 2. Set up watsonx.ai Runtime service and API key

1. Create a [watsonx.ai Runtime](https://cloud.ibm.com/catalog/services/watsonxai-runtime) service instance (choose the Lite plan, which is a free instance).

2. Generate an [API Key](https://dataplatform.cloud.ibm.com/docs/content/wsj/analyze-data/ml-authentication.html). 

3. Associate the watsonx.ai Runtime service to the project that you created in [watsonx.ai](https://dataplatform.cloud.ibm.com/docs/content/wsj/getting-started/assoc-services.html?context=cpdaas). 

## Step 3. Install and import relevant libraries and set up your credentials

We need a few libraries and modules for this tutorial. Make sure to import the following ones; if they're not installed, you can resolve this issue with a quick pip installation.

In [ ]:
# Install required packages
!pip install -q image ibm-watsonx-ai

In [ ]:
# Required imports
import getpass, os, base64, json
from ibm_watsonx_ai import Credentials
from ibm_watsonx_ai.foundation_models import ModelInference
from PIL import Image

To set our credentials, we need the `WATSONX_APIKEY` and `WATSONX_PROJECT_ID` you generated in step 1. We will also set the URL serving as the API endpoint.

In [ ]:
WATSONX_APIKEY = getpass.getpass("Please enter your watsonx.ai Runtime API key (hit enter): ")

WATSONX_PROJECT_ID = getpass.getpass("Please enter your project ID (hit enter): ")

URL = "https://us-south.ml.cloud.ibm.com"

We can use the `Credentials` class to encapsulate our passed credentials.

In [ ]:
credentials = Credentials(
    url=URL,
    api_key=WATSONX_APIKEY
)

## Step 4. Set up the API request for the LLM

The `augment_api_request_body` function takes the user query and image as parameters and augments the body of the API request. We will use this function in each iteration of inferencing the LLM.


In [ ]:
def augment_api_request_body(user_query, image):
    messages = [
        {
            "role": "user",
            "content": [{
                "type": "text",
                "text": user_query
            },
            {
                "type": "image_url",
                "image_url": {
                    "url": f"data:image/jpeg;base64,{image}"
                }
            }]
        }
    ]
    
    return messages

We can also instantiate the model interface by using the `ModelInference` class. 

In [ ]:
model = ModelInference(
        model_id="meta-llama/llama-4-scout-17b-16e-instruct",
        credentials=credentials,
        project_id=WATSONX_PROJECT_ID,
        params={
            "max_tokens": 128000,
            "temperature": 0       
        }
    )

## Step 5. Encode images

To encode our images in a way that is digestible for the LLM, we encode them to bytes that we then decode to UTF-8 representation. In this case, our images are located in the local `images` directory. You can find sample images in the AI personal trainer directory in our [GitHub repository](https://github.com/IBM/ibmdotcom-tutorials).

In [ ]:
directory = "images"  #directory name
images = []
filenames = []

for filename in os.listdir(directory): 
  if filename.endswith(".jpeg") or filename.endswith(".png"): 
    filepath = directory + '/' +filename
    with  open(filepath, "rb") as f:
      images.append(base64.b64encode(f.read()).decode('utf-8')) 
    filenames.append(filename)
    print(filename)

AdobeStock_91307353_Preview.jpeg
AdobeStock_922896276_Preview.jpeg
AdobeStock_311431811_Preview.jpeg
AdobeStock_13628669_Preview.jpeg
AdobeStock_263663835_Preview.jpeg
AdobeStock_389847104_Preview.jpeg
AdobeStock_74024563_Preview.jpeg
AdobeStock_211228969_Preview.jpeg
AdobeStock_94122326_Preview.jpeg
AdobeStock_169219024_Preview.jpeg
AdobeStock_86254899_Preview.jpeg


## Step 6. Categorize input with the Llama 4 model

Now that we have loaded and encoded our images, we can query the Vision model. Our prompt is specific to our desired output to limit the model's creativity as we seek valid JSON output. We will store the description, category and workout type of each image in a list called `image_descriptions`. 

In [ ]:
user_query = """Provide a description, category, and workout type for the kinds of exercise equipment in each image, eg. "barbell", "dumbbell", "machine", "bodyweight", etc.   

                Classify the description as "equipment" or "other".
                Classify the category as "barbell", "dumbbell", "machine", "bodyweight", etc.
                Classify the workout type as "strength", "endurance", "flexibility", "balance", "cardio", etc.
                
                Ensure the output is valid JSON. Do not create new categories or occasions. Only use the allowed classifications.
                
                Your response should be in this schema: 
                {
                    "description": "<description>",
                    "category": "<category>",
                    "workout_type": "<workout_type>"
                }
                """

image_descriptions = []

for i in range(len(images)):
    image = images[i]
    message = augment_api_request_body(user_query, image)
    response = model.chat(messages=message)
    result = response['choices'][0]['message']['content']
    print(result)
    image_descriptions.append(result)

```json
{
    "description": "yoga mat",
    "category": "other",
    "workout_type": "flexibility"
}
```
```json
{
    "description": "Stairmaster",
    "category": "machine",
    "workout_type": "cardio"
}
```
```json
{
    "description": "Pilates reformer",
    "category": "machine",
    "workout_type": "strength"
}
```
{
    "description": "elliptical trainer",
    "category": "machine",
    "workout_type": "cardio"
}
```json
{
    "description": "A ballet barre",
    "category": "barre",
    "workout_type": "strength"
}
```
```json
{
    "description": "A dumbbell with multiple weight plates",
    "category": "dumbbell",
    "workout_type": "strength"
}
```
```json
{
    "description": "treadmill",
    "category": "machine",
    "workout_type": "cardio"
}
```
```
{
  "description": "exercise bike",
  "category": "machine",
  "workout_type": "cardio"
}
```
```json
{
    "description": "barbell",
    "category": "barbell",
    "workout_type": "strength"
}
```
```json
{
    "descript

## Step 7. Generate workout plans with the LLM

Now that we have each clothing and shoe item categorized, it will be much easier for the reasoning model to generate an outfit for the selected occasion. Let's instantiate and query the reasoning model.

In [ ]:
reasoning_model = ModelInference(
        model_id="meta-llama/llama-4-scout-17b-16e-instruct",
        credentials=credentials,
        project_id=WATSONX_PROJECT_ID
)

To align the filenames with the image descriptions, we can enumerate the list of image descriptions and create a list of dictionaries. These dictionaries will store the description, category, occasion and filename of each item in the respective fields. 

In [ ]:
# Add filenames to the image descriptions
gym_equipment = []
for i, desc in enumerate(image_descriptions):
    # Clean up the string by removing markdown code block markers and 'json' identifier
    cleaned_desc = desc.strip()
    if cleaned_desc.startswith('```'):
        cleaned_desc = cleaned_desc.split('```')[1]  # Remove opening ```
        if cleaned_desc.startswith('json'):
            cleaned_desc = cleaned_desc[4:]  # Remove 'json' identifier
        cleaned_desc = cleaned_desc.split('```')[0]  # Remove closing ```
    cleaned_desc = cleaned_desc.strip()
    
    desc_dict = json.loads(cleaned_desc)
    desc_dict['filename'] = filenames[i]
    image_descriptions[i] = json.dumps(desc_dict)

gym_equipment = [json.loads(js) for js in image_descriptions]
print(gym_equipment)

[{'description': 'yoga mat', 'category': 'other', 'workout_type': 'flexibility', 'filename': 'AdobeStock_91307353_Preview.jpeg'}, {'description': 'Stairmaster', 'category': 'machine', 'workout_type': 'cardio', 'filename': 'AdobeStock_922896276_Preview.jpeg'}, {'description': 'Pilates reformer', 'category': 'machine', 'workout_type': 'strength', 'filename': 'AdobeStock_311431811_Preview.jpeg'}, {'description': 'elliptical trainer', 'category': 'machine', 'workout_type': 'cardio', 'filename': 'AdobeStock_13628669_Preview.jpeg'}, {'description': 'A ballet barre', 'category': 'barre', 'workout_type': 'strength', 'filename': 'AdobeStock_263663835_Preview.jpeg'}, {'description': 'A dumbbell with multiple weight plates', 'category': 'dumbbell', 'workout_type': 'strength', 'filename': 'AdobeStock_389847104_Preview.jpeg'}, {'description': 'treadmill', 'category': 'machine', 'workout_type': 'cardio', 'filename': 'AdobeStock_74024563_Preview.jpeg'}, {'description': 'exercise bike', 'category': 'm

Now, let's query the Llama 4 model to produce a workout plan for our specified criteria by using the `gym_equipment` list. The following sample below uses the provided input: 
```python
workout_type = "strength"
length = "1 hour"
fitness_level = "beginner"
```

In [ ]:
workout_type = input("Enter the workout type")             #strength, endurance, flexibility, balance, cardio, etc. (e.g. "strength")
length = input("Enter the length of the workout")          #30 minutes, 1 hour, 1.5 hours, etc.
fitness_level = input("Enter your fitness level")          #beginner, intermediate or advanced
prompt = f"""Use the description, category, and workout type of the exercise equipment in my gym to put together a workout for a {fitness_level} {workout_type} workout. The workout must be no longer than {length}.
                You must inclue the filename of each image in your output along with the file extension. Here is the equipment in my gym: {gym_equipment}"""

messages = [
        {
            "role": "user",
            "content": [
                {
                    "type": "text",
                    "text": f"{prompt}"
                }
            ]
        }
    ]

workout = reasoning_model.chat(messages=messages)['choices'][0]['message']['content']
print(workout)

Based on the equipment available in your gym, I've put together a beginner-friendly strength workout that can be completed within an hour. This workout will target various muscle groups and include a mix of exercises to get a good overall strength training session.

**Warm-up (5 minutes)**

* Start with some light cardio to get your blood flowing and warm up your muscles. You can use the treadmill or the exercise bike for this. For example, you can jog on the treadmill for 5 minutes or pedal on the exercise bike at a moderate pace.

**Strength Training Circuit (45 minutes)**

1. **Barbell Squats** (3 sets of 8-12 reps)
Use the barbell and weightlifting bench with a barbell rack and weights (AdobeStock_169219024_Preview.jpeg). Stand with your feet shoulder-width apart, then bend your knees and lower your body down until your thighs are parallel to the ground. 
2. **Dumbbell Chest Press** (3 sets of 8-12 reps)
 Lie on your back with your knees bent and feet flat on the floor, holding a d

Great! This personal training algorithm returned a well-described strength training plan including the number of repetitions, or reps, and rest time. The LLM also places importance on proper form and hydration.

With this generated workout description, we can also display the gym equipment that the model recommends. To do so, we can simply extract the filenames. In case the model mentions the same filename twice, it is important to check whether the image has not already been displayed as we iterate the list of images. We can do so by storing displayed images in the `selected_items` list. Finally, we can display the selected items. 

In [ ]:
selected_items = []
#extract the images of clothing that the model recommends
for item, uploaded_file in zip(gym_equipment, images):
    if item['filename'].lower() in workout.lower() and not any(key['filename'] == item['filename'] for key in selected_items):
        selected_items.append({
            'image': uploaded_file,
            'category': item['category'],
            'filename': item['filename']
        })

#display the selected clothing items
if len(selected_items) > 0:
    for item in selected_items:
        display(Image.open(directory + '/' + item['filename']))
        

## Conclusion

In this tutorial, you built a fitness app that uses an AI coach to customize and automate training sessions for new clients. This coaching platform and other advancements alike have the potential to reshape the fitness industry by providing real-time feedback to a real person looking for online training. Using photos or screenshots of the user's equipment, workout plans are customized by the AI tool to meet the specified criteria. Workouts for weight loss, muscle gain and strength training are all possible outputs. The Llama 4 model was critical for labeling and categorizing each item as well as generating the workout plan.

Some next steps for building off this application can include:
- Creating a frontend to allow for AI chatbot functionality, similar to ChatGPT.
- Building an app for users to use the AI personal trainer on-the-go. This might be an iOS app for iPhone users, for example.
- Redesigning the use case to meet demand for other services such as mental health support. 